In [5]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from modules import optimize_null, forward_simulation, construct_null_trajectory
from modules.elbo import ELBO

In [26]:
# load behavioral data for erratic and smooth videos from AXB task
f_path = '../data/jongmin_data/'

n_corr_obs_erratic = pd.read_csv(Path(f_path) / 'nErratic.csv', header=None).to_numpy()
n_total_obs_erratic = pd.read_csv(Path(f_path) / 'mErratic.csv', header=None).to_numpy()

n_corr_obs_smooth = pd.read_csv(Path(f_path) / 'nSmooth.csv', header=None).to_numpy()
n_total_obs_smooth = pd.read_csv(Path(f_path) / 'mSmooth.csv', header=None).to_numpy()

### Estimate curvature for erratic videos

In [32]:
# estimate curvature for erratic video
n_dim = 5
elbo_erratic = ELBO(n_dim, n_corr_obs_erratic, n_total_obs_erratic, n_starts=10, n_iterations=60000)
x_erratic, p_erratic, errors_erratic, kl_loss_erratic, ll_loss_erratic, c_prior_erratic, d_prior_erratic, l_prior_erratic, c_post_erratic, d_post_erratic, l_post_erratic, c_est_erratic = elbo.optimize_ELBO_SGD()

Running MLE to initialize posterior..........................
Current loss: 958.7406005859375
Loss updated
Iteration 1 | Loss: 958.7406005859375
Current loss: 775.6856689453125
Loss updated
Iteration 2 | Loss: 775.6856689453125
Current loss: 824.144775390625
Iteration 3 | Loss: 824.144775390625
Current loss: 789.619873046875
Iteration 4 | Loss: 789.619873046875
Current loss: 790.868408203125
Iteration 5 | Loss: 790.868408203125
Current loss: 920.6142578125
Iteration 6 | Loss: 920.6142578125
Current loss: 935.5322265625
Iteration 7 | Loss: 935.5322265625
Current loss: 903.9589233398438
Iteration 8 | Loss: 903.9589233398438
Current loss: 940.296630859375
Iteration 9 | Loss: 940.296630859375
Current loss: 805.6762084960938
Iteration 10 | Loss: 805.6762084960938
Epoch: 0, Loss: 3415.8214136868914
Epoch: 250, Loss: 2072.6606587186734
Epoch: 500, Loss: 1680.5937510501124
Epoch: 750, Loss: 1476.6648920226908
Epoch: 1000, Loss: 1341.7462733943678
Epoch: 1250, Loss: 1236.6241849631458
Epoch: 15

In [53]:
est_global_curvature_erratic = torch.rad2deg(elbo_erratic.mu_prior_c.detach())
print(f'Estimated global curvature (erratic): {est_global_curvature_erratic} degrees')
print(f'Average estimated local curvature from posterior (erratic): {torch.rad2deg(torch.mean(elbo_erratic.mu_post_c).detach())} degrees')
print(f'Average estimated local curvature from most likely trajectory (erratic): {torch.rad2deg(torch.mean(c_est_erratic).detach())} degrees')

Estimated global curvature (erratic): 76.16986083984375 degrees
Average estimated local curvature from posterior (erratic): 76.17062890237396 degrees
Average estimated local curvature from most likely trajectory (erratic): 76.17062377929688 degrees


In [69]:
# compute standard error of the mean (SEM)
sem_cest_erratic = torch.sqrt(torch.var(c_est_erratic) / c_est_erratic.shape[1])
sem_post_erratic = torch.sqrt(torch.var(elbo_erratic.mu_post_c) / elbo_erratic.mu_post_c.shape[0])
sem_prior_erratic = elbo_erratic.sigma_prior_c

print(f'SEM estimated from curvature of most likely trajectory (erratic): {sem_cest_erratic}')
print(f'SEM estimated from posterior (erratic): {sem_post_erratic}')
print(f'SEM estimated from prior (erratic): {sem_prior_erratic.detach().numpy()[0]}')

SEM estimated from curvature of most likely trajectory (erratic): 0.0034754432272166014
SEM estimated from posterior (erratic): 0.0034754551567432665
SEM estimated from prior (erratic): 0.07731721733869623


### Estimate curvature for smooth videos

In [70]:
# estimate curvature for smooth video
n_dim = 5
elbo_smooth = ELBO(n_dim, n_corr_obs_smooth, n_total_obs_smooth, n_starts=10, n_iterations=60000)
x_smooth, p_smooth, errors_smooth, kl_loss_smooth, ll_loss_smooth, c_prior_smooth, d_prior_smooth, l_prior_smooth, c_post_smooth, d_post_smooth, l_post_smooth, c_est_smooth = elbo_smooth.optimize_ELBO_SGD()

Running MLE to initialize posterior..........................
Current loss: 973.8453369140625
Loss updated
Iteration 1 | Loss: 973.8453369140625
Current loss: 885.8164672851562
Loss updated
Iteration 2 | Loss: 885.8164672851562
Current loss: 1041.803955078125
Iteration 3 | Loss: 1041.803955078125
Current loss: 925.2678833007812
Iteration 4 | Loss: 925.2678833007812
Current loss: 914.5169677734375
Iteration 5 | Loss: 914.5169677734375
Current loss: 948.547607421875
Iteration 6 | Loss: 948.547607421875
Current loss: 913.183349609375
Iteration 7 | Loss: 913.183349609375
Current loss: 942.5897827148438
Iteration 8 | Loss: 942.5897827148438
Current loss: 943.4119262695312
Iteration 9 | Loss: 943.4119262695312
Current loss: 917.407470703125
Iteration 10 | Loss: 917.407470703125
Epoch: 0, Loss: 8804.34472586646
Epoch: 250, Loss: 3636.7629255686243
Epoch: 500, Loss: 2637.3925784163052
Epoch: 750, Loss: 2212.9377630261693
Epoch: 1000, Loss: 1931.921828450646
Epoch: 1250, Loss: 1757.376380814831

In [71]:
est_global_curvature_smooth = torch.rad2deg(elbo_smooth.mu_prior_c.detach())
print(f'Estimated global curvature (smooth): {est_global_curvature_smooth} degrees')
print(f'Average estimated local curvature from posterior (smooth): {torch.rad2deg(torch.mean(elbo_smooth.mu_post_c).detach())} degrees')
print(f'Average estimated local curvature from most likely trajectory (smooth): {torch.rad2deg(torch.mean(c_est_smooth).detach())} degrees')

Estimated global curvature (smooth): 48.2387809753418 degrees
Average estimated local curvature from posterior (smooth): 48.24112972777927 degrees
Average estimated local curvature from most likely trajectory (smooth): 48.24113464355469 degrees


In [73]:
# compute standard error of the mean (SEM)
sem_cest_smooth = torch.sqrt(torch.var(c_est_smooth) / c_est_smooth.shape[1])
sem_post_smooth = torch.sqrt(torch.var(elbo_smooth.mu_post_c) / elbo_smooth.mu_post_c.shape[0])
sem_prior_smooth = elbo_smooth.sigma_prior_c

print(f'SEM estimated from curvature of most likely trajectory (smooth): {sem_cest_smooth}')
print(f'SEM estimated from posterior (smooth): {sem_post_smooth}')
print(f'SEM estimated from prior (smooth): {sem_prior_smooth.detach().numpy()[0]}')

SEM estimated from curvature of most likely trajectory (smooth): 0.06210042163729668
SEM estimated from posterior (smooth): 0.06210041081173285
SEM estimated from prior (smooth): 0.3323638423112401
